# Cleaning data for visualization
While the data is not committed to the repository to save space, each of the data sets used in this notebook may be accessed at the following link, downloaded and stored locally in the `data` subdirectory of the `propublica_hotspots` directory such that you can run this notebook:
- [ProPublica: Toxic Air Pollution Hot Spots](https://www.propublica.org/datastore/dataset/toxic-air-pollution-hot-spots)
- [2020 DEC Redistricting Data Race Table, geographically filtered by Place](https://data.census.gov/cedsci/table?g=0100000US%241600000&y=2020&d=DEC%20Redistricting%20Data%20%28PL%2094-171%29&tid=DECENNIALPL2020.P1)
- [2010 DEC Redistricting Data Race Table, geographically filtered by Block](https://data.census.gov/cedsci/table?g=0400000US01%241000000,02%241000000,04%241000000,05%241000000,06%241000000,08%241000000,09%241000000,10%241000000,11%241000000,12%241000000,13%241000000,15%241000000,16%241000000,17%241000000,18%241000000,19%241000000,20%241000000,21%241000000,22%241000000,23%241000000,24%241000000,25%241000000,26%241000000,27%241000000,28%241000000,29%241000000,30%241000000,31%241000000,32%241000000,33%241000000,34%241000000,35%241000000,36%241000000,37%241000000,38%241000000,39%241000000,40%241000000,41%241000000,42%241000000,44%241000000,45%241000000,46%241000000,47%241000000,48%241000000,49%241000000,50%241000000,51%241000000,53%241000000,54%241000000,55%241000000,56%241000000&d=DEC%20Redistricting%20Data%20%28PL%2094-171%29&tid=DECENNIALPL2010.P1)

Resources:
- [EPA's explanation of the components of the Risk-Screening Environmental Indicators (RSEI) Model](https://www.epa.gov/rsei/ways-get-rsei-results): EPA data set that ProPublica used to form its grids.
- [EPA's Risk-Screening Environmental Indicators (RSEI) Methodology](https://www.epa.gov/sites/default/files/2020-02/documents/rsei_methodology_v2.3.8.pdf): documentation for the RSEI data set, useful for understanding how grids are represented.
- [RSEI data dictionary for crosswalk data set](https://www.epa.gov/rsei/rsei-data-dictionary-census-crosswalks)
- [ProPublica: The Most Detailed Map of Cancer-Causing Industrial Air Pollution in the U.S.](https://projects.propublica.org/toxmap/): Original visualization created using the data in this project, good way to explore the data and get a familiarity with it.


In [1]:
import pandas as pd
import json
import numpy as np

## Loading pollution data

In [2]:
data = json.load(open("../data/toxmaps_files_2022-03-15b/toxmaps_files_for_data_store/hotspot_perimeters_for_data_store.geojson"))
data = data["features"]
pollution_hotspot_df = pd.DataFrame([d["properties"] for d in data])

# Normalizing with EPA acceptable risk, which is 1/10,000
pollution_hotspot_df["avg_ilcr"] = pollution_hotspot_df["avg_ilcr"] * 10000
pollution_hotspot_df["max_ilcr"] = pollution_hotspot_df["max_ilcr"] * 10000

pollution_hotspot_df


,id,place,state,avg_ilcr,max_ilcr,area,pop
0,24228,Corpus Christi,Texas,0.294000,3.403039,0.006039,57150
1,27627,Catoosa,Oklahoma,0.213759,0.472415,0.000526,68
2,38309,Chicago,Illinois,0.252534,0.252534,0.000071,388
3,27293,Attica,Indiana,0.280931,0.280931,0.000069,5
4,1180,Watertown,Wisconsin,0.313408,0.418059,0.000145,872
...,...,...,...,...,...,...,...
1356,38536,Monaca,Pennsylvania,0.183536,0.183536,0.000070,0
1357,8056,Prairie du Sac,Wisconsin,0.178760,0.178760,0.000073,280
1358,209,Sheboygan,Wisconsin,0.175148,0.175148,0.000073,210
1359,9940,Hooksett,New Hampshire,0.186700,0.186700,0.000072,36


## Loading census data

In [3]:
name_keys = ["city", "CDP", "town", "municipality", "borough", "village", "corporation", "unified government", "urban county", "consolidated government", "consolidated government \(balance\)", "metropolitan government \(balance\)", "metro government \(balance\)", "\(balance\)"]

def extract_city_name(x):
    item_found = False
    i = 0

    while not item_found and i < len(name_keys):
        if name_keys[i] in x:
            item_found = True
        else:
            i = i + 1

    if item_found:
        return x.split(" " + name_keys[i])[0]
    else:
        return x

In [4]:
raw_census_df = pd.read_csv("../data/DECENNIALPL2020.P1_2022-07-19T162239/DECENNIALPL2020.P1_data_with_overlays_2022-04-27T142004.csv", skiprows=1)
raw_census_df

,id,Geographic Area Name,!!Total:,!!Total:!!Population of one race:,!!Total:!!Population of one race:!!White alone,!!Total:!!Population of one race:!!Black or African American alone,!!Total:!!Population of one race:!!American Indian and Alaska Native alone,!!Total:!!Population of one race:!!Asian alone,!!Total:!!Population of one race:!!Native Hawaiian and Other Pacific Islander alone,!!Total:!!Population of one race:!!Some Other Race alone,...,!!Total:!!Population of two or more races:!!Population of four races:!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:,!!Total:!!Population of two or more races:!!Population of five races:!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander,!!Total:!!Population of two or more races:!!Population of five races:!!White; Black or African American; American Indian and Alaska Native; Asian; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of six races:,!!Total:!!Population of two or more races:!!Population of six races:!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race
0,1600000US0100100,"Abanda CDP, Alabama",133,132,95,34,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
1,1600000US0100124,"Abbeville city, Alabama",2358,2275,1165,1039,5,15,0,51,...,0,0,0,0,0,0,0,0,0,0
2,1600000US0100460,"Adamsville city, Alabama",4366,4221,1741,2313,31,10,3,123,...,0,0,0,0,0,0,0,0,0,0
3,1600000US0100484,"Addison town, Alabama",659,643,624,0,9,2,1,7,...,0,0,0,0,0,0,0,0,0,0
4,1600000US0100676,"Akron town, Alabama",225,220,19,199,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31904,1600000US7287638,"Voladoras comunidad, Puerto Rico",670,290,136,8,2,1,0,143,...,0,0,0,0,0,0,0,0,0,0
31905,1600000US7287863,"Yabucoa zona urbana, Puerto Rico",5196,3052,924,458,36,1,4,1629,...,0,0,0,0,0,0,0,0,0,0
31906,1600000US7288035,"Yauco zona urbana, Puerto Rico",13569,7455,2887,558,91,12,2,3905,...,0,0,0,0,0,0,0,0,0,0
31907,1600000US7288121,"Yaurel comunidad, Puerto Rico",769,598,18,112,2,1,0,465,...,0,1,0,1,0,0,0,0,0,0


In [5]:
census_df = pd.DataFrame(raw_census_df)
census_df.columns = census_df.columns.str.strip()

# Filtering out unneeded columns
cols = [col for col in census_df.columns if "!!Total:!!Population of one race:!!" in col or col in ["id", "Geographic Area Name", "!!Total:"]]
census_df = census_df.loc[:, cols]

# Renaming columns
census_df.columns = [s.replace("!!Total:!!Population of one race:!!", "") for s in census_df.columns]
census_df = census_df.rename(columns={"!!Total:": "Total"})

# Splitting geographic area name into city and state (seperated by comma)
location_name_split = census_df["Geographic Area Name"].str.split(",", expand=True)
census_df = census_df.drop("Geographic Area Name", axis=1)
census_df["City"] = location_name_split[0]
census_df["State"] = location_name_split[1].str.strip()

# Filling in the state of places where the county is also stored in the geographic area name
census_df.loc[~location_name_split[2].isna(), "State"] = location_name_split[2].str.strip()

# Filtering out Puerto Rico
census_df = census_df.loc[census_df["State"] != "Puerto Rico"]

# Extracting city names without place type indicator
census_df["City"] = census_df["City"].apply(lambda x: extract_city_name(x))

census_df

,id,Total,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,City,State
0,1600000US0100100,133,95,34,0,0,0,3,Abanda,Alabama
1,1600000US0100124,2358,1165,1039,5,15,0,51,Abbeville,Alabama
2,1600000US0100460,4366,1741,2313,31,10,3,123,Adamsville,Alabama
3,1600000US0100484,659,624,0,9,2,1,7,Addison,Alabama
4,1600000US0100676,225,19,199,0,2,0,0,Akron,Alabama
...,...,...,...,...,...,...,...,...,...,...
31612,1600000US5684852,118,105,1,0,2,0,0,Woods Landing-Jelm,Wyoming
31613,1600000US5684925,4773,3944,6,66,22,3,313,Worland,Wyoming
31614,1600000US5685015,1644,1447,0,18,0,0,86,Wright,Wyoming
31615,1600000US5686665,131,123,0,0,0,0,0,Yoder,Wyoming


## Merging data sets

In [6]:
df = census_df.merge(pollution_hotspot_df, how="left", left_on=["City", "State"], right_on=["place", "state"])
df

,id_x,Total,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,City,State,id_y,place,state,avg_ilcr,max_ilcr,area,pop
0,1600000US0100100,133,95,34,0,0,0,3,Abanda,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1600000US0100124,2358,1165,1039,5,15,0,51,Abbeville,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1600000US0100460,4366,1741,2313,31,10,3,123,Adamsville,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1600000US0100484,659,624,0,9,2,1,7,Addison,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1600000US0100676,225,19,199,0,2,0,0,Akron,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31870,1600000US5684852,118,105,1,0,2,0,0,Woods Landing-Jelm,Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31871,1600000US5684925,4773,3944,6,66,22,3,313,Worland,Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31872,1600000US5685015,1644,1447,0,18,0,0,86,Wright,Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31873,1600000US5686665,131,123,0,0,0,0,0,Yoder,Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Block data retrieval

In [7]:
# Using Hawaii as example/proof of concept
hi_blocks_raw = pd.read_csv("../data/HI/DECENNIALSF12010.P5_2022-07-29T131133/DECENNIALSF12010.P5_data_with_overlays_2022-07-29T131128.csv", skiprows=1)
hi_blocks_raw

,id,Total,Total!!Not Hispanic or Latino,Total!!Not Hispanic or Latino!!White alone,Total!!Not Hispanic or Latino!!Black or African American alone,Total!!Not Hispanic or Latino!!American Indian and Alaska Native alone,Total!!Not Hispanic or Latino!!Asian alone,Total!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Total!!Not Hispanic or Latino!!Some Other Race alone,Total!!Not Hispanic or Latino!!Two or More Races,Total!!Hispanic or Latino,Total!!Hispanic or Latino!!White alone,Total!!Hispanic or Latino!!Black or African American alone,Total!!Hispanic or Latino!!American Indian and Alaska Native alone,Total!!Hispanic or Latino!!Asian alone,Total!!Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Total!!Hispanic or Latino!!Some Other Race alone,Total!!Hispanic or Latino!!Two or More Races,Geographic Area Name
0,1000000US150010201001000,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Block 1000, Block Group 1, Census Tract 201, H..."
1,1000000US150010201001001,123,114,59,0,0,27,4,1,23,9,3,0,1,0,0,0,5,"Block 1001, Block Group 1, Census Tract 201, H..."
2,1000000US150010201001002,16,16,9,0,0,1,2,0,4,0,0,0,0,0,0,0,0,"Block 1002, Block Group 1, Census Tract 201, H..."
3,1000000US150010201001003,11,8,4,0,0,4,0,0,0,3,0,0,0,0,0,1,2,"Block 1003, Block Group 1, Census Tract 201, H..."
4,1000000US150010201001004,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Block 1004, Block Group 1, Census Tract 201, H..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25011,1000000US150099912000002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Block 0002, Block Group 0, Census Tract 9912, ..."
25012,1000000US150099912000003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Block 0003, Block Group 0, Census Tract 9912, ..."
25013,1000000US150099912000004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Block 0004, Block Group 0, Census Tract 9912, ..."
25014,1000000US150099912000005,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Block 0005, Block Group 0, Census Tract 9912, ..."


In [8]:
def clean_blocks(raw_blocks):
    blocks = pd.DataFrame(raw_blocks)

    # Cleaning columns
    blocks["Total"] = blocks["Total"].astype(str)
    blocks["Total"] = blocks["Total"].str.split("(").str[0]
    blocks["Total"] = blocks["Total"].astype(int)

    # Filtering out unneeded columns
    blocks = blocks.drop("Geographic Area Name", axis=1)

    # Renaming columns
    blocks.columns = [col.replace("Total!!Not Hispanic or Latino!!", "")
                            .replace("Total!!Hispanic or Latino!!", "Hispanic or Latino: ")
                            .replace("Total!!", "") for col in blocks.columns]

    # Formatting ID for use in crosswalk
    blocks["id"] = blocks["id"].str.replace("1000000US", "")
    blocks["id"] = pd.to_numeric(blocks["id"])

    return blocks

In [9]:
hi_blocks = clean_blocks(hi_blocks_raw)
hi_blocks

,id,Total,Not Hispanic or Latino,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Two or More Races,Hispanic or Latino,Hispanic or Latino: White alone,Hispanic or Latino: Black or African American alone,Hispanic or Latino: American Indian and Alaska Native alone,Hispanic or Latino: Asian alone,Hispanic or Latino: Native Hawaiian and Other Pacific Islander alone,Hispanic or Latino: Some Other Race alone,Hispanic or Latino: Two or More Races
0,150010201001000,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,150010201001001,123,114,59,0,0,27,4,1,23,9,3,0,1,0,0,0,5
2,150010201001002,16,16,9,0,0,1,2,0,4,0,0,0,0,0,0,0,0
3,150010201001003,11,8,4,0,0,4,0,0,0,3,0,0,0,0,0,1,2
4,150010201001004,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25011,150099912000002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
25012,150099912000003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
25013,150099912000004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
25014,150099912000005,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Crosswalk retrieval
This is the file that the EPA uses to match census-designated blocks with its grid map coordinates used in the measuring of pollution levels.

In [10]:
hi_x_walk_raw = pd.read_csv("../data/HI/CensusBlock2010_Hawaii_810m.csv")
hi_x_walk_raw

,GridID,X,Y,BlockID00,UR,PCT_B_C,PCT_C_B,PCT_CP_B
0,34,254,-203,150010212022293,Z,0.547465,0.028747,0.000000
1,34,254,-202,150010212022293,Z,0.451824,0.023725,0.000000
2,34,257,-203,150010212022359,Z,0.649448,0.145474,0.000000
3,34,257,-202,150010212022359,Z,0.350130,0.078428,0.000000
4,34,258,-202,150010212022359,Z,0.000463,0.000104,0.000000
...,...,...,...,...,...,...,...,...
109999,34,115,58,150090315023000,Z,0.242369,0.068603,0.087691
110000,34,135,48,150090308002023,Z,0.797387,0.097005,0.000000
110001,34,136,48,150090308002023,Z,0.192064,0.023365,0.000000
110002,34,136,49,150090308002023,Z,0.010631,0.001293,0.000000


In [11]:
def add_race_to_crosswalk(raw_crosswalk, blocks_df):
    # Makes copy of raw DataFrame
    crosswalk = pd.DataFrame(raw_crosswalk)

    race_cols = [col for col in blocks_df.columns if col != "id"]
    
    # Getting number of total and white-identifying in each block
    crosswalk = crosswalk.merge(blocks_df, left_on="BlockID00", right_on="id")
    crosswalk = crosswalk.drop(columns="id")

    # Normalizing based on percentage
    for col in race_cols:
        crosswalk[col] = crosswalk[col] * crosswalk["PCT_B_C"]

    return crosswalk

In [12]:
add_race_to_crosswalk(hi_x_walk_raw, hi_blocks)

,GridID,X,Y,BlockID00,UR,PCT_B_C,PCT_C_B,PCT_CP_B,Total,Not Hispanic or Latino,...,Some Other Race alone,Two or More Races,Hispanic or Latino,Hispanic or Latino: White alone,Hispanic or Latino: Black or African American alone,Hispanic or Latino: American Indian and Alaska Native alone,Hispanic or Latino: Asian alone,Hispanic or Latino: Native Hawaiian and Other Pacific Islander alone,Hispanic or Latino: Some Other Race alone,Hispanic or Latino: Two or More Races
0,34,254,-203,150010212022293,Z,0.547465,0.028747,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
1,34,254,-202,150010212022293,Z,0.451824,0.023725,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
2,34,257,-203,150010212022359,Z,0.649448,0.145474,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
3,34,257,-202,150010212022359,Z,0.350130,0.078428,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
4,34,258,-202,150010212022359,Z,0.000463,0.000104,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109999,34,115,58,150090315023000,Z,0.242369,0.068603,0.087691,117.064227,107.854205,...,0.242369,11.148974,9.210022,1.454214,0.0,0.0,0.484738,0.0,5.574487,1.696583
110000,34,135,48,150090308002023,Z,0.797387,0.097005,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
110001,34,136,48,150090308002023,Z,0.192064,0.023365,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
110002,34,136,49,150090308002023,Z,0.010631,0.001293,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


## Creating DataFrame for grid layout
Each row corresponds to a grid in which the EPA measured pollution levels.

In [13]:
def create_grid_from_crosswalk(crosswalk: pd.DataFrame) -> pd.DataFrame:
    # Columns (roughly) reflect the blocks .csv file
    grid = pd.DataFrame(columns=["X", "Y"])

    # Get list of unique cell coordinates from crosswalk
    unique_cell_points = crosswalk[["X", "Y"]].value_counts().reset_index(name="count")

    grid["X"] = unique_cell_points["X"]
    grid["Y"] = unique_cell_points["Y"]
    
    return grid

In [14]:
create_grid_from_crosswalk(hi_x_walk_raw)

,X,Y
0,-34,100
1,-32,99
2,-38,102
3,-51,106
4,-30,98
...,...,...
40352,219,-90
40353,219,-89
40354,219,-86
40355,219,-77


In [15]:
def create_race_grid(raw_crosswalk: pd.DataFrame, raw_blocks: pd.DataFrame) -> pd.DataFrame:
    cleaned_blocks = clean_blocks(raw_blocks)

    # Retrieve crosswalk with the proportion of total and white residents contributed by the block to the grid space
    crosswalk_with_race = add_race_to_crosswalk(raw_crosswalk, cleaned_blocks)

    grid = create_grid_from_crosswalk(crosswalk_with_race)

    # Calculate total and white-identifying in grid based on number in block, percentage of block that is contained in each grid
    grid_race_counts = crosswalk_with_race.groupby(["X", "Y"]).sum().reset_index()
    race_cols = [col for col in cleaned_blocks.columns if col != "id"]
    grid = grid.merge(grid_race_counts.loc[:, ["X", "Y"] + race_cols], on=["X", "Y"])

    # Calculate percentages for each race category
    for col in race_cols:
        if col != "Total":
            grid[col + " %"] = grid[col] / grid["Total"]

    return grid

In [16]:
grid = create_race_grid(hi_x_walk_raw, hi_blocks_raw)
grid

,X,Y,Total,Not Hispanic or Latino,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,...,Some Other Race alone %,Two or More Races %,Hispanic or Latino %,Hispanic or Latino: White alone %,Hispanic or Latino: Black or African American alone %,Hispanic or Latino: American Indian and Alaska Native alone %,Hispanic or Latino: Asian alone %,Hispanic or Latino: Native Hawaiian and Other Pacific Islander alone %,Hispanic or Latino: Some Other Race alone %,Hispanic or Latino: Two or More Races %
0,-34,100,5913.056076,5589.219405,706.151138,59.195532,10.530401,3361.968086,473.037915,3.069885,...,0.000519,0.164934,0.054766,0.011332,0.000856,0.000465,0.004968,0.002478,0.008743,0.025924
1,-32,99,3420.049070,3233.910956,458.390648,24.173781,3.136054,1888.490476,230.897156,4.787988,...,0.001400,0.182464,0.054426,0.011188,0.001075,0.000389,0.005093,0.003536,0.008889,0.024256
2,-38,102,3052.800517,2896.250179,920.882966,61.395952,3.363095,1454.989868,112.055564,5.291862,...,0.001733,0.110807,0.051281,0.018875,0.000765,0.000543,0.005732,0.001917,0.005863,0.017585
3,-51,106,540.051498,493.757213,411.077669,17.113910,2.000000,22.502089,2.031952,0.000000,...,0.000000,0.072274,0.085722,0.057849,0.001852,0.001852,0.001881,0.011125,0.005522,0.005641
4,-30,98,2610.817266,2519.673381,320.130205,14.039737,0.223853,1610.929648,98.204736,4.065934,...,0.001557,0.180817,0.034910,0.006504,0.000206,0.000000,0.008764,0.001742,0.004022,0.013672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40352,219,-90,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40353,219,-89,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40354,219,-86,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40355,219,-77,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Retrieving pollution data by grid space

In [17]:
data = json.load(open("../data/toxmaps_files_2022-03-15b/toxmaps_files_for_data_store/hotspot_gridsquares_for_data_store.geojson"))
data = data["features"]

rows = []

for d in data:
    long_centerpoint = sum([d["geometry"]["coordinates"][0][i][0] for i in range(4)]) / 4
    lat_centerpoint = sum([d["geometry"]["coordinates"][0][i][1] for i in range(4)]) / 4

    centerpoint = (lat_centerpoint, long_centerpoint)

    row = d["properties"]
    row["centerpoint"] = centerpoint

    rows.append(row)

pollution_gridspace_df = pd.DataFrame(rows)
pollution_gridspace_df = pollution_gridspace_df.rename(columns={"x": "X", "y": "Y"})

# Normalizing with EPA acceptable risk, which is 1/10,000
pollution_gridspace_df["ilcr"] = pollution_gridspace_df["ilcr"] * 10000

pollution_gridspace_df

,X,Y,gridcode,pop,ilcr,ilcr_2014,ilcr_2015,ilcr_2016,ilcr_2017,ilcr_2018,cluster_id,centerpoint
0,251,966,14,38,0.420968,4.790455e-05,3.769349e-05,6.004251e-05,0.000032,0.000033,26,"(30.097545326241, -93.8879279214834)"
1,91,892,14,1926,0.121234,1.414638e-05,1.198967e-05,1.055006e-05,0.000010,0.000014,6,"(29.574912519332347, -95.23914509756315)"
2,263,985,14,11,0.161599,1.559376e-05,1.456587e-05,2.335146e-05,0.000013,0.000014,26,"(30.234148583816776, -93.78325946023887)"
3,648,977,14,338,2.323963,2.435303e-04,2.340715e-04,3.008869e-04,0.000257,0.000127,2,"(30.062467554026572, -90.54709637204613)"
4,261,979,14,47,0.244414,2.325287e-05,2.111681e-05,3.412669e-05,0.000021,0.000022,26,"(30.190724935572675, -93.80128060364268)"
...,...,...,...,...,...,...,...,...,...,...,...,...
41183,-113,750,14,0,0.175297,9.806828e-06,9.468954e-06,7.979511e-06,0.000012,0.000048,8087,"(28.53472346994222, -96.93312965135505)"
41184,241,911,14,0,0.116320,1.193648e-05,1.036187e-05,1.696981e-05,0.000009,0.000009,26,"(29.697678817620076, -93.98187490836554)"
41185,233,978,14,48,0.240806,2.288500e-05,2.135100e-05,3.396695e-05,0.000021,0.000022,26,"(30.187903120807775, -94.03726140364626)"
41186,1094,1976,14,135,0.155266,1.076361e-07,2.090732e-05,2.132226e-05,0.000021,0.000014,680,"(37.02493370273622, -85.92636006604992)"


## Alabama case study

In [18]:
al_blocks_raw = pd.read_csv("../data/AL/DECENNIALSF12010.P5_2022-07-29T112605/DECENNIALSF12010.P5_data_with_overlays_2022-07-29T112432.csv", skiprows=1)
al_blocks_raw

c:\Users\Jason D'Amico\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,Total,Total!!Not Hispanic or Latino,Total!!Not Hispanic or Latino!!White alone,Total!!Not Hispanic or Latino!!Black or African American alone,Total!!Not Hispanic or Latino!!American Indian and Alaska Native alone,Total!!Not Hispanic or Latino!!Asian alone,Total!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Total!!Not Hispanic or Latino!!Some Other Race alone,Total!!Not Hispanic or Latino!!Two or More Races,Total!!Hispanic or Latino,Total!!Hispanic or Latino!!White alone,Total!!Hispanic or Latino!!Black or African American alone,Total!!Hispanic or Latino!!American Indian and Alaska Native alone,Total!!Hispanic or Latino!!Asian alone,Total!!Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone,Total!!Hispanic or Latino!!Some Other Race alone,Total!!Hispanic or Latino!!Two or More Races,Geographic Area Name
0,1000000US010010201001000,61,61,55,4,0,1,0,0,1,0,0,0,0,0,0,0,0,"Block 1000, Block Group 1, Census Tract 201, A..."
1,1000000US010010201001001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Block 1001, Block Group 1, Census Tract 201, A..."
2,1000000US010010201001002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Block 1002, Block Group 1, Census Tract 201, A..."
3,1000000US010010201001003,75,71,66,4,0,0,0,0,1,4,0,0,0,0,0,4,0,"Block 1003, Block Group 1, Census Tract 201, A..."
4,1000000US010010201001004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Block 1004, Block Group 1, Census Tract 201, A..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252261,1000000US011339659003101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Block 3101, Block Group 3, Census Tract 9659, ..."
252262,1000000US011339659003102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Block 3102, Block Group 3, Census Tract 9659, ..."
252263,1000000US011339659003103,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Block 3103, Block Group 3, Census Tract 9659, ..."
252264,1000000US011339659003104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Block 3104, Block Group 3, Census Tract 9659, ..."


In [19]:
cont_x_walk_raw = pd.read_csv("../data/CensusBlock2010_ConUS_810m.csv")
cont_x_walk_raw

,GridID,X,Y,BlockID00,UR,PCT_B_C,PCT_C_B,PCT_CP_B
0,14,-2126,1523,40270111071059,Z,0.006029,0.000754,0.000000
1,14,-2125,1523,40270111071059,Z,0.993539,0.124338,0.000000
2,14,-2125,1522,40270111072002,Z,0.431253,0.219350,0.000000
3,14,-2125,1523,40270111072002,Z,0.441065,0.224341,0.000000
4,14,-2124,1522,40270111072002,Z,0.002394,0.001218,0.000000
...,...,...,...,...,...,...,...,...
41612389,14,-866,2897,560459511001785,Z,0.029898,0.075755,0.000000
41612390,14,-810,2896,560459513003023,Z,1.000000,0.000313,0.000000
41612391,14,-847,2936,560459511001065,Z,1.000000,0.009000,0.272847
41612392,14,-810,2897,560459513001015,Z,0.979167,0.002615,0.000000


In [20]:
al_grid = create_race_grid(cont_x_walk_raw, al_blocks_raw)
al_grid

,X,Y,Total,Not Hispanic or Latino,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,...,Some Other Race alone %,Two or More Races %,Hispanic or Latino %,Hispanic or Latino: White alone %,Hispanic or Latino: Black or African American alone %,Hispanic or Latino: American Indian and Alaska Native alone %,Hispanic or Latino: Asian alone %,Hispanic or Latino: Native Hawaiian and Other Pacific Islander alone %,Hispanic or Latino: Some Other Race alone %,Hispanic or Latino: Two or More Races %
0,1035,1482,411.912900,405.766233,21.020359,376.270580,2.460665,0.000000,0.0,0.0,...,0.000000,0.014602,0.014922,0.002428,0.010067,0.0,0.000000,0.0,0.002428,0.0
1,1035,1481,1143.021901,1114.305927,53.729173,1046.875416,0.536457,0.000000,0.0,1.0,...,0.000875,0.010643,0.025123,0.004374,0.011992,0.0,0.000000,0.0,0.008756,0.0
2,1036,1482,723.120458,705.572521,20.577286,679.527128,1.000000,0.000000,0.0,0.0,...,0.000000,0.006179,0.024267,0.005532,0.008837,0.0,0.000000,0.0,0.009899,0.0
3,1036,1483,495.929274,495.279686,3.317756,486.365985,1.152975,0.000000,0.0,0.0,...,0.000000,0.008959,0.001310,0.000000,0.000655,0.0,0.000000,0.0,0.000655,0.0
4,937,1084,580.401346,567.534888,308.349776,249.358991,1.693364,1.108809,0.0,0.0,...,0.000000,0.012102,0.022168,0.014031,0.000140,0.0,0.000562,0.0,0.007434,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208186,998,1225,7.106532,7.106532,7.056486,0.050046,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
208187,998,1224,7.106532,7.106532,7.056486,0.050046,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
208188,998,1223,7.106532,7.106532,7.056486,0.050046,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
208189,998,1222,7.106532,7.106532,7.056486,0.050046,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0


**NOTE:** In the grid below, the "Total" field and the "pop" field are each supposed to represent the total number of people residing in the grid space. The "Total" field is sourced from the 2010 census translation onto the grid, and the "pop" field is sourced from the ProPublica data. As can be seen, the two values are roughly on the same magnitude, but are not equivalent. According to page 13 of the [RSEI methodology document](https://www.epa.gov/sites/default/files/2020-02/documents/rsei_methodology_v2.3.8.pdf), the source of ProPublica's grid pollution data: 

> Population values for non-decennial years are estimated based on linear interpolations at
the block level between the 1990 and 2000 and between the 2000 and 2010 U.S. census
datasets, and on extrapolation back to 1988 and forward to 2018.

While it is unclear which year the "pop" field in the ProPublica data represents in the documentation they provided, it is likely that it uses a similar extrapolation method (and the ProPublica data dictionary also notes that the "pop" field on the grid space level may be extrapolated). Due to the limited information on how to improve the accuracy of the "Total" field to be more comparable to the "pop" field, we can move forward in analysis despite this discrepancy. 

In [21]:
al_grid = al_grid.merge(pollution_gridspace_df, how="left", on=["X", "Y"])

# Filling empty grid spaces for cancer rates to be 0
cancer_cols = list(pollution_gridspace_df.columns[pollution_gridspace_df.columns.str.startswith("ilcr")])
al_grid[cancer_cols] = al_grid[cancer_cols].fillna(value=0)

al_grid

,X,Y,Total,Not Hispanic or Latino,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,...,gridcode,pop,ilcr,ilcr_2014,ilcr_2015,ilcr_2016,ilcr_2017,ilcr_2018,cluster_id,centerpoint
0,1035,1482,411.912900,405.766233,21.020359,376.270580,2.460665,0.000000,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,1035,1481,1143.021901,1114.305927,53.729173,1046.875416,0.536457,0.000000,0.0,1.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,1036,1482,723.120458,705.572521,20.577286,679.527128,1.000000,0.000000,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,1036,1483,495.929274,495.279686,3.317756,486.365985,1.152975,0.000000,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,937,1084,580.401346,567.534888,308.349776,249.358991,1.693364,1.108809,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208186,998,1225,7.106532,7.106532,7.056486,0.050046,0.000000,0.000000,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
208187,998,1224,7.106532,7.106532,7.056486,0.050046,0.000000,0.000000,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
208188,998,1223,7.106532,7.106532,7.056486,0.050046,0.000000,0.000000,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
208189,998,1222,7.106532,7.106532,7.056486,0.050046,0.000000,0.000000,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [23]:
al_grid.to_csv("../data/al_grid.csv", index=False)